# Lab | APIS

In this lab, you will collect historical data about the Nobel Prize winners using [this free and non-authenticated API](https://www.nobelprize.org/organization/developer-zone-2/). According to the documentation available [here](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1#/default/get_nobelPrizes). The base url is: "http://api.nobelprize.org/2.1/" followed by a string to specify what kind of information do you want to retrieve. The acceptable options are:

* nobelPrices
* nobelPrice/category/year
* laureates
* laureate/laureateID

# Getting the information using requests

Use the Python `requests`, and `json` libraries to obtain the information of ALL the Nobel Prizes. Make sure to verify that you get the proper status code (200).

The json outputs are simple plain text that need to be converted into the corresponding nested dictionary. Use the `.json()` method to cast the output into a Python dictionary.

Use the Pandas library to collect all the information into a Panda's DataFrame.

In [2]:
!pip install requests

In [4]:
import requests
import json
import pandas as pd

url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
        print("All good!")
        print("==============")
        print("\n")
        nobels = response.json()

All good!




# Processing the output

Process the Pandas DataFrame in order to have only the following columns:

- category
- dateAwarded (as DateTime in "yyyy-mm-dd" format)
- prizeAmount
- prizeAmountAdjusted
- Number_of_laureates
- motivation
- laureate_ids (as a list)

In [34]:
url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
    prizes_list = response.json()['nobelPrizes']
    # Your code here


In [38]:
url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
    prizes_list = response.json()['nobelPrizes']
    list(prizes_list)
    df = pd.DataFrame(prizes_list)
    df_2 = pd.DataFrame({
        'category': df['category'].apply(lambda x: x['en']),  # Extract the 'en' (English) field
        'dateAwarded': pd.to_datetime(df['dateAwarded']),  # Convert to DateTime
        'prizeAmount': df['prizeAmount'],
        'prizeAmountAdjusted': df['prizeAmountAdjusted'],
        'Number_of_laureates': df['laureates'].apply(lambda x: len(x) if isinstance(x, list) else 0),  # Count number of laureates
        'motivation': df['laureates'].apply(lambda x: "; ".join([l['motivation']['en'] for l in x if 'motivation' in l]) if isinstance(x, list) else ""), #Combine motivations
        'laureate_ids': df['laureates'].apply(lambda x: [l['id'] for l in x] if isinstance(x, list) else [])  # Extract laureate IDs as a list
    })
    
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

df_2

,category,dateAwarded,prizeAmount,prizeAmountAdjusted,Number_of_laureates,motivation,laureate_ids
0,Chemistry,1901-11-12,150782,10531894,1,in recognition of the extraordinary services h...,[160]
1,Literature,1901-11-14,150782,10531894,1,in special recognition of his poetic compositi...,[569]
2,Peace,1901-12-10,150782,10531894,2,for his humanitarian efforts to help wounded s...,"[462, 463]"
3,Physics,1901-11-12,150782,10531894,1,in recognition of the extraordinary services h...,[1]
4,Physiology or Medicine,1901-10-30,150782,10531894,1,"for his work on serum therapy, especially its ...",[293]
...,...,...,...,...,...,...,...
665,Economic Sciences,2023-10-09,11000000,11000000,1,for having advanced our understanding of women...,[1034]
666,Literature,2023-10-05,11000000,11000000,1,for his innovative plays and prose which give ...,[1032]
667,Peace,2023-10-06,11000000,11000000,1,for her fight against the oppression of women ...,[1033]
668,Physics,2023-10-03,11000000,11000000,3,for experimental methods that generate attosec...,"[1026, 1027, 1028]"


# Getting a Pandas DataFrame with the details of awarded authors/institutions

If you dive deeper and use the API to retrieve the details of some laureate_ids, you will notice that not allways the Nobel Prize was awarded to individuals. In some cases, the awards were given to institutions.

Get the unique ids from the previous datasets and prepare the following functions:

- get_name(laureate) ( it should return the english name 'fullName' of the individual or 'orgName' of the institution )

- get_gender(laureate) ( it should return the gender or 'Unknown' for individuals, and 'None' for institutions )

- get_birthdate(laureate) ( it should return the birthdate when it's avaialble or 'Unknown' otherwise )

- get_age(laureate) ( it should return the age of the awarded individual or 'Unknown' when it's not avaialble or for institutions )

- get_city(laureate) ( it should return the english name of the city when it's available or 'Unknown' otherwise )

- get_country(laureate) ( it should return the english name of the country when it's available or 'Unknown' otherwise )

- get_continent(laureate) ( it should return the english name of the continent when it's available or 'Unknown' otherwise )

- get_latitude(laureate) ( it should return the city's latitude when it's available or 'Unknown' otherwise )

- get_longitude(laureate) ( it should return the city's longitude
 when it's available or 'Unknown' otherwise )

Create the following dictionaries:

```python
laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}                        

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}
```

For each unique `laureate_id` of the previous DataFrame make an API call to get the details of the awarded individual/intitution and iterate of the previous dictionaries keys in order to add the corresponding information of each `laureate_id` in the empty lists of `laureates_dict`.

Finally, create a Pandas DataFrame named `laureates_df` using the `laureates_dict`.

In [ ]:
import time
from tqdm import tqdm


ids = [int(item) for l in df['laureate_ids'].values for item in l]
unique_ids = set(ids)

def get_name(laureate):
    if 'fullName' in laureate:
        return laureate['fullName'].get('en', 'Unknown')
    elif 'orgName' in laureate:
        return laureate['orgName'].get('en', 'Unknown')
    return 'Unknown'

def get_gender(laureate):
    if 'gender' in laureate:
        return laureate['gender']
    return 'None' if 'orgName' in laureate else 'Unknown'

def get_birthdate(laureate):
    birth_date = "Unknown"
    if 'birth' in laureate and 'date' in laureate['birth']:
        birth_date = laureate['birth']['date']
    return birth_date

def get_age(laureate):
    birth_date = "Unknown"
    award_date = "Unknown"
    birth_date = get_birthdate(laureate)
    if birth_date != "Unknown":
        birth_date = datetime.strptime(birth_date, '%Y-%m-%d')
        age = (datetime.now() - birth_date).days // 365
        return age
    return "Unknown"

def get_city(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('city', {}).get('en', 'Unknown')
    return 'Unknown'

def get_country(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('country', {}).get('en', 'Unknown')
    return 'Unknown'

def get_continent(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('continent', {}).get('en', 'Unknown')
    return 'Unknown'

def get_latitude(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('location', {}).get('latitude', 'Unknown')
    return 'Unknown'

def get_longitude(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('location', {}).get('longitude', 'Unknown')
    return 'Unknown'


laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}

for index, id in enumerate(tqdm(unique_ids)):
    url = "https://api.nobelprize.org/2/laureate/" + str(id)
    response = requests.get(url)
    if response.status_code == 200:
        laureate = response.json()['laureates'][0]

        laureates_dict["ID"].append(id)
        for key, func in functions_dict.items():
            if func:  # Only call functions for keys that are not "ID"
                laureates_dict[key].append(func(laureate))

laureates_df = pd.DataFrame(laureates_dict)

laureates_df



100%|██████████| 992/992 [10:16<00:00,  1.61it/s]


,ID,Name,Gender,Birth_date,Age,City,Country,Continent,Latitude,Longitude
0,1,Wilhelm Conrad Röntgen,male,1845-03-27 00:00:00,None,Lennep,Germany,Europe,51.178742,7.189696
1,2,Hendrik Antoon Lorentz,male,1853-07-18 00:00:00,None,Arnhem,the Netherlands,Europe,51.984257,5.910857
2,3,Pieter Zeeman,male,1865-05-25 00:00:00,None,Zonnemaire,the Netherlands,Europe,51.713056,3.951111
3,4,Antoine Henri Becquerel,male,1852-12-15 00:00:00,None,Paris,France,Europe,48.860093,2.355954
4,5,Pierre Curie,male,1859-05-15 00:00:00,None,Paris,France,Europe,48.860093,2.355954
...,...,...,...,...,...,...,...,...,...,...
987,1030,Louis E. Brus,male,Unknown,None,"Cleveland, OH",USA,North America,41.496386,-81.710675
988,1031,Aleksey Yekimov,male,Unknown,None,Leningrad,Russia,Europe,59.956651,30.333547
989,1032,Jon Fosse,male,1959-09-29 00:00:00,None,Haugesund,Norway,Europe,59.410150,5.275511
990,1033,Narges Mohammadi,female,1972-04-21 00:00:00,None,Zanjan,Iran,Asia,36.666667,48.483333


In [42]:
import requests
import pandas as pd
from datetime import datetime
import time
from tqdm import tqdm


ids = [int(item) for l in df_2['laureate_ids'].values for item in l]
unique_ids = set(ids)

def get_name(laureate):
    if 'fullName' in laureate:
        return laureate['fullName'].get('en', 'Unknown')
    elif 'orgName' in laureate:
        return laureate['orgName'].get('en', 'Unknown')
    return 'Unknown'

def get_gender(laureate):
    if 'gender' in laureate:
        return laureate['gender']
    return 'None' if 'orgName' in laureate else 'Unknown'

def get_birthdate(laureate):
    birth_date = "Unknown"
    if 'birth' in laureate and 'date' in laureate['birth']:
        birth_date = laureate['birth']['date']
    return birth_date

def get_age(laureate):
    #birth_date = "Unknown"
    #award_date = "Unknown"
    birth_date = get_birthdate(laureate)
    if birth_date != "Unknown":
        birth_date = datetime.strptime(birth_date, '%Y-%m-%d')
        age = (datetime.now() - birth_date).days // 365
        return age
    return "Unknown"

def get_city(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('city', {}).get('en', 'Unknown')
    return 'Unknown'

def get_country(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('country', {}).get('en', 'Unknown')
    return 'Unknown'

def get_continent(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('continent', {}).get('en', 'Unknown')
    return 'Unknown'

def get_latitude(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('location', {}).get('latitude', 'Unknown')
    return 'Unknown'

def get_longitude(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('location', {}).get('longitude', 'Unknown')
    return 'Unknown'


laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}



In [50]:
for index, id in enumerate(tqdm(unique_ids)):
    url = "https://api.nobelprize.org/2/laureate/" + str(id)
    response = requests.get(url)
    if response.status_code == 200:
        laureate = response.json()['laureates'][0]

        laureates_dict["ID"].append(id)
        for key, func in functions_dict.items():
            if func:  # Only call functions for keys that are not "ID"
                laureates_dict[key].append(func(laureate))

laureates_df = pd.DataFrame(laureates_dict)

laureates_df


  0%|          | 0/992 [00:00<?, ?it/s]


TypeError: list indices must be integers or slices, not str

# Country ranking

Get a ranking countries by the number of times that they had been awarded in any category.

In [48]:
# Step 1: Explode the list of laureate IDs to get one row per laureate
df_exploded = df_2.explode('laureate_ids')

# Step 2: Create a dictionary to store the country counts
country_counts = {}

# Step 3: Loop through each unique laureate_id to get the country information
for laureate_id in df_exploded['laureate_ids'].unique():
    url = f"https://api.nobelprize.org/2.1/laureate/{laureate_id}"
    response = requests.get(url)

    if response.status_code == 200:
        laureate = response.json()#['laureates'][0]
        country = get_country(laureate)  # Using the previously defined get_country function

        if country in country_counts:
            country_counts[country] += 1
        else:
            country_counts[country] = 1

# Step 4: Convert the dictionary to a DataFrame and sort it by the count
country_ranking_df = pd.DataFrame(list(country_counts.items()), columns=['Country', 'Awards'])
country_ranking_df = country_ranking_df.sort_values(by='Awards', ascending=False).reset_index(drop=True)

# Display the ranking
print(country_ranking_df)

   Country  Awards
0  Unknown     993


In [ ]:
# Your code here

,ID
Country,
USA,296
United Kingdom,91
Germany,84
France,63
Russia,30
...,...
Greece,1
Ghana,1
Faroe Islands (Denmark),1
